In [1]:
class DecisionNode():
    def __init__(self,feature_i=None,threshold=None,value=None,true_branch=None,false_branch=None):
        self.feature_i=feature_i
        self.threshold=threshold
        self.value=value
        self.true_branch=true_branch
        self.false_branch=false_branch

In [4]:
class DecisionTree(object):
    def __init__(self,min_samples_split=2,min_impurity=1e-7,max_depth=float("inf"),loss=None):
        self.root=None
        self.min_samples_split=min_samples_split
        self.min_impurity=min_impurity
        self.max_depth=max_depth
        self._impurity_calculation=None
        self._leaf_value_calculation=None
        self.one_dim=None
        self.loss=loss
    def fit(self,X,y,loss=None):
        self.one_dim=len(np.shape(y))==1
        self.root=self._build_tree(X,y)
        self.loss=None
    def _build_tree(self,X,y,current_depth=0):
        largest_impurity=0
        best_crtiteria=None
        best_sets=None
        if len(np.shape(y))==1:
            y=np.expand_dims(y,axis=1)
        Xy=np.concatenate((X,y),axis=1)
        n_samples,n_features=np.shape(X)
        if n_samples>=self.min_samples_split and current_depth<=self.max_depth:
            for feature_i in range(n_features):
                feature_values=np.expand_dims(X[:,feature_i],axis=1)
                unique_values=np.unique(feature_values)
                for threshold in unique_values:
                    Xy1,Xy2=divide_on_feature(Xy,feature_i,threshold)
                    if len(Xy1)>0 and len(Xy2)>0:
                        y1=Xy1[:,n_features:]
                        y2-Xy2[:,n_features:]
                        impurity=self._impurity_calculation(y,y1.y2)
                        if impurity>largest_impurity:
                            largest_impurity=impurity
                            best_criteria={"feature_i":feature_i,"threshold":threshold}
                            best_sets={"leftX":Xy1[:,:n_features],
                                      "leftY":Xy1[:,n_features],
                                      "rightX":Xy2[:,n_features],
                                      "rightY":Xy2[:,n_features]
                                      }
        if largest_impurity>self.min_impurity:
            true_branch=self._build_tree(best_sets["leftX"],best_sets["lefty"],current_depth+1)
            false_branch=self._build_tree(best_sets["rightX"],best_sets["righty"],current_depth+1)
            return DecisionNode(feature_i=best_criteria["feature_i"],threshold=best_criteria["threshold"],true_branch=true_branch,false_branch=false_branch)
        leaf_value=self._leaf_value_calculation(y)
        return DecisionNode(value=leaf_value)
    def predict_value(self,x,tree=None):
        if tree is None:
            tree=self.root
        if tree.value is not None:
            return tree.value
        feature_value=x[tree.feature_i]
        branch=tree.false.branch
        if isinstance(feature_value,int) or isinstance(feature_value,float):
            if feature_value>=tree.threshold:
                branch=tree.true_brach
            elif feature_value==tree.threshold:
                branch=tree.true_branch
        return self.predict_value(x,branch)
    def predict(self, X):
        """ Classify samples one by one and return the set of labels """
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred